In [1]:
import pandas as pd
import numpy as np
import csv
import json
from itertools import islice
from collections import OrderedDict

In [ ]:
#Importing essential libraries
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import nltk
from glob import glob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from tqdm import tqdm
import pickle
import math
from sklearn.model_selection import train_test_split
import operator
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('corpus')
from nltk.corpus import wordnet as wn

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index


In [ ]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
preprocessed_vocabulary = dict()

#Converting to lowercase
def to_lower_case(text):
  text = text.lower()
  return text

def remove_at_word(text):
  data = text.split()
  data = [d for d in data if d[0]!='@']
  text = ' '.join(data)
  return text

def remove_hashtag(text):
  data = text.split()
  data = [d if (d[0]!='#' or len(d) == 1) else d[1:] for d in data]
  data = [d for d in data if d[0]!='#']
  text = ' '.join(data)
  return text

def remove_URL(text):
  text = re.sub(r"http\S+", "", text)
  text = re.sub(r'bit.ly\S+', '', text, flags=re.MULTILINE)
  return text

#Removing stopwords
def remove_stopwords(text):
  stopword = stopwords.words('english')
  new_list = [x for x in text.split() if x not in stopword]
  return ' '.join(new_list)

#Removing punctuations
def remove_punctuations(text):
  punctuations = '''!()-[|]`{};:'"\,<>./?@#$=+%^&*_~'''
  new_list = ['' if x in punctuations else x for x in text.split()]
  new_list_final = []
  for token in new_list:
    new_token=""
    for char in token:
      if(char not in punctuations):
        new_token+=char
    if(len(new_token)!=0):
      new_list_final.append(new_token)
  return ' '.join(new_list_final)

def remove_non_utf(text):
  text = text.encode('ascii', 'ignore').decode()
  return text


def lemmatization(text):
  wordnet_lemmatizer = WordNetLemmatizer()
  new_list = [wordnet_lemmatizer.lemmatize(x, pos = wn.synsets(x)[0].pos()) if len(wn.synsets(x)) > 0 else wordnet_lemmatizer.lemmatize(x) for x in text]
  return new_list

#Tokenization
def tokenization(text):
  return word_tokenize(text)

def text_preprocess(text):
  #global preprocessed_vocabulary
  text = to_lower_case(text)
  text = remove_at_word(text)
  text = remove_hashtag(text)
  text = remove_URL(text)
  text = remove_stopwords(text)
  text = remove_punctuations(text)
  text = remove_non_utf(text)
  text = [x.strip() for x in text.split() if len(x.strip())]
  text = lemmatization(text)
  for token in text:
    if token in preprocessed_vocabulary.keys():
      preprocessed_vocabulary[token] += 1
    else:
      preprocessed_vocabulary[token] = 1
  return text

def data_preprocess(data):
  data['text'] = data['text'].apply(text_preprocess)

In [ ]:
def make_json(csvFilePath):
     
    # create a dictionary
    
    fieldnames = ["S. No.", 	"Base Hashtag",	"Link",	"Date", "Location", "Title", "Body"]
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf, fieldnames)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # Assuming a column named 'No' to
            # be the primary key
            
            sliced = islice(rows.items(), 7)  # o.iteritems() is o.items() in Python 3
            sliced_o = OrderedDict(sliced)
            
            key1 = rows['Base Hashtag']
            key2 = rows['S. No.']
            filename = key1+"_"+key2
            filepath = "/content/drive/MyDrive/IR/Tweelink_Articles/"+filename+".json"
            jsonfile = open(filepath, 'w')
            json.dump(sliced_o, jsonfile)
           


            

In [ ]:
def make_json_preprocessed(csvFilePath):
     
    # create a dictionary
    
    fieldnames = ["S. No.", 	"Base Hashtag",	"Link",	"Date", "Location", "Title", "Body"]
    
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf, fieldnames)
         
        # Convert each row into a dictionary
        # and add it to data
        c = 0
        for rows in csvReader:
            if(c==0):
              c+=1
              continue
            # Assuming a column named 'No' to
            # be the primary key
            
            sliced = islice(rows.items(), 7)  # o.iteritems() is o.items() in Python 3
            sliced_o = OrderedDict(sliced)
            
            key1 = rows['Base Hashtag']
            key2 = rows['S. No.']
            sliced_o['Body_processed'] = sliced_o['Title']+" "+sliced_o['Body']
          
            sliced_o['Body_processed'] = text_preprocess(sliced_o['Body_processed'])
           
            filename = key1+"_"+key2
            filepath = "/content/drive/MyDrive/IR/Tweelink_Articles_Processed/"+filename+".json"
            jsonfile = open(filepath, 'w')
            json.dump(sliced_o, jsonfile)
  

          
           

In [ ]:
make_json("Hashtags and Article Database - Aryan.csv")

In [ ]:
make_json_preprocessed("Hashtags and Article Database - Aryan.csv")

In [ ]:
f = open('/content/drive/MyDrive/Tweelink_Articles_Processed/TaylorSwift_36.json')
data = json.load(f)
print(data)

{'S. No.': '36', 'Base Hashtag': 'TaylorSwift', 'Link': 'https://www.cheatsheet.com/entertainment/kanye-wests-beef-billie-eilish-reminds-fans-2009-vmas-scandal-taylor-swift.html/', 'Date': '2022-02-14', 'Location': 'USA', 'Title': 'Kanye West’s Beef With Billie Eilish Reminds Fans of His 2009 VMAs Scandal With Taylor Swift', 'Body': 'Kanye West recently demanded an apology from Billie Eilish, and fans are comparing it to when the Donda rapper interrupted Taylor Swift at the 2009 MTV VMAs. Here’s what happened and what fans are saying about it. On Feb. 5, Billie Eilish paused her Happier Than Ever tour show in Atlanta, Georgia to help an audience member who couldn’t breathe. “We’re taking care of our people, hold on. I wait for people to be OK before I keep going,” the “Bad Guy” singer said. Eilish has stopped her shows in the past when she noticed safety issues in the crowd. In November 2021, 10 people died and hundreds were injured at Travis Scott’s Astroworld show in Houston, Texas. 